In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [2]:
bs,bptt = 64,70

## Language modeling

### Data

In [3]:
re_br = re.compile('<br />')
def sub_br(x): return re_br.sub("\n", x)
import spacy
spacy_en = spacy.load('en')

def spacy_tok(x): return [tok.text for tok in spacy_en.tokenizer(sub_br(x))]

In [4]:
PATH='data/aclImdb/'
%ls {PATH}

aclImdb_v1.tar.gz  imdbEr.txt  imdb.vocab  models/  README  test/  tmp/  train/


In [5]:
TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'

In [6]:
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

In [72]:
%ls {TRN} | head

0_0.txt
0_3.txt
0_9.txt
10000_0.txt
10000_4.txt
10000_8.txt
1000_0.txt
10001_0.txt
10001_10.txt
10001_4.txt
ls: write error


In [73]:
!cat {TRN}1000_0.txt

I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop-socky fung-ku, but what I got instead was a comedy. So, it wasn't quite was I was expecting, but I really liked it anyway! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them!! I was laughing my ass off. I mean, the cops were just so bad! And when I say bad, I mean The Shield Vic Macky bad. But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose...man, oh man. What can you say about that hottie. She was great and put those other actresses to shame. She should work more often!!!!! I also really liked the fight scene outside of the building. That was done really well. Lots of fighting and people getting their heads banged up. FUN! Last, but not least Joe Estevez and William Smith were great as the...well, I wasn't sure what they were, but they seemed to be having fun and throwing out l

In [74]:
!find {TRN} -name '*.txt' | xargs cat | wc -w

17486581


In [75]:
!find {TRN} -name '*.txt' | xargs cat | wc -w

17486581


In [7]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [8]:
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [9]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text), len(TEXT.vocab)

(4602, 34945, 1, 20621966, 34945)

In [77]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'it', 'in']

In [78]:
md.trn_ds[0].text[:12], next(iter(md.trn_dl))

(['i',
  'just',
  'do',
  "n't",
  'get',
  'some',
  'of',
  'the',
  'big',
  'premises',
  'of',
  'this'],
 (Variable containing:
      12    687     13  ...    1923     19     17
      51    615     25  ...      10     63     51
      53      3      3  ...      42    428    833
          ...            ⋱           ...         
      51     64      3  ...     111    544      6
       6      2   2075  ...      90     43    552
    3343   4351      5  ...      35      2    249
  [torch.cuda.LongTensor of size 61x64 (GPU 0)], Variable containing:
      51
     615
      25
    ⋮   
      93
     406
      43
  [torch.cuda.LongTensor of size 3904 (GPU 0)]))

### Train

In [7]:
em_sz = 200
nh = 500
nl = 3
clip=0.3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
reg_fn=partial(seq2seq_reg, alpha=2, beta=1)

In [12]:
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
learner = md.get_model(opt_fn, bs, em_sz, nh, nl, pad_token=pad_idx,
                       dropouti=0.1, dropout=0.1, wdrop=0.2, dropoute=0.04, dropouth=0.1)

In [13]:
learner.fit(1e-3, 1, wds=1e-6, reg_fn=reg_fn, clip=clip)

A Jupyter Widget

[ 0.      4.9619  4.767 ]                                     



In [14]:
learner.fit(3e-3, 3, wds=1e-6, reg_fn=reg_fn, cycle_len=3, clip=clip, cycle_save_name='adam3_5')

A Jupyter Widget

[ 0.      4.8114  4.6071]                                     
[ 1.      4.6945  4.486 ]                                     
[ 2.      4.6423  4.4497]                                     
[ 3.      4.7243  4.5014]                                     
[ 4.      4.6335  4.4207]                                     
[ 5.      4.58    4.3916]                                     
[ 6.      4.6804  4.462 ]                                     
[ 7.      4.6023  4.3915]                                     
[ 8.      4.5517  4.3639]                                     



In [43]:
#learner.load_cycle('adam3_5', 0)

In [15]:
learner.save_encoder('adam3_5_enc')

In [16]:
learner.fit(3e-3, 4, wds=1e-6, reg_fn=reg_fn, cycle_len=10, clip=clip, cycle_save_name='adam3_10')

A Jupyter Widget

[ 0.      4.6732  4.4673]                                     
[ 1.      4.661   4.4494]                                     
[ 2.      4.6346  4.4297]                                     
[ 3.      4.6071  4.4054]                                     
[ 4.      4.587   4.3785]                                     
[ 5.      4.5641  4.3529]                                     
[ 6.      4.5196  4.329 ]                                     
[ 7.      4.5075  4.3142]                                     
[ 8.      4.4983  4.3066]                                     
[ 9.      4.476   4.3053]                                     
[ 10.       4.6526   4.4288]                                  
[ 11.       4.6265   4.4146]                                  
[ 12.       4.6127   4.4013]                                  
[ 13.       4.5894   4.3774]                                  
[ 14.       4.5545   4.3523]                                  
[ 15.       4.5439   4.3298]                           

In [17]:
learner.save_encoder('adam3_10_enc')

In [18]:
learner.fit(1e-3, 1, wds=1e-6, reg_fn=reg_fn, cycle_len=10, clip=clip)

A Jupyter Widget

[ 0.      4.5049  4.2964]                                     
[ 1.      4.5034  4.2941]                                     
[ 2.      4.4958  4.2872]                                     
[ 3.      4.4764  4.281 ]                                     
[ 4.      4.4659  4.2718]                                     
[ 5.      4.4744  4.2662]                                     
[ 6.      4.4406  4.2579]                                     
[ 7.      4.4416  4.254 ]                                     
[ 8.      4.4424  4.2525]                                     
[ 9.      4.4321  4.2521]                                     



In [19]:
learner.fit(3e-3, 1, wds=1e-6, reg_fn=reg_fn, cycle_len=20, clip=clip, cycle_save_name='adam3_20')

A Jupyter Widget

[ 0.      4.4307  4.2544]                                     
[ 1.      4.6123  4.3939]                                     
[ 2.      4.6016  4.3855]                                     
[ 3.      4.5948  4.3822]                                     
[ 4.      4.5919  4.3739]                                     
[ 5.      4.5717  4.365 ]                                     
[ 6.      4.5613  4.3543]                                     
[ 7.      4.5619  4.343 ]                                     
[ 8.      4.5389  4.3319]                                     
[ 9.      4.5297  4.3192]                                     
[ 10.       4.5267   4.3073]                                  
[ 11.       4.4985   4.2934]                                  
[ 12.       4.4856   4.2833]                                  
[ 13.       4.475    4.2729]                                  
[ 14.       4.4706   4.2643]                                  
[ 15.       4.4527   4.2565]                           

In [26]:
learner.save_encoder('adam3_20_enc')

In [20]:
math.exp(4.25)

70.10541234668786

### Test

In [21]:
m=learner.model
s=""". So, it wasn't quite was I was expecting, but I really liked it anyway! The best"""
s = [spacy_tok(s)]
t=TEXT.numericalize(s)
' '.join(s[0])

". So , it was n't quite was I was expecting , but I really liked it anyway ! The best"

In [23]:
m[0].bs=1
m.eval()
m.reset()
res,*_ = m(t)
m[0].bs=bs

In [24]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['of',
 'movie',
 'performance',
 'scene',
 'film',
 'part',
 'actor',
 'friend',
 'thing',
 '<unk>']

In [25]:
for i in range(20):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))

of all , i was n't expecting much from this movie , but i was n't expecting much . i 

### Sentiment

In [14]:
topm = learner.model[0]

In [27]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))
IMDB_TEXT = data.Field(lower=True, tokenize=spacy_tok)#, fix_length=sl)#, batch_first=True)
IMDB_TEXT.vocab = TEXT.vocab
pickle.dump(IMDB_TEXT, open(f'{PATH}models/IMDB_TEXT.pkl','wb'))

In [8]:
IMDB_TEXT = pickle.load(open(f'{PATH}models/IMDB_TEXT.pkl','rb'))

In [9]:
def postproc(arr,vocab,train): return [1 if o==1 else 0 for o in arr]

pad_idx = IMDB_TEXT.vocab.stoi[IMDB_TEXT.pad_token]
IMDB_LABEL = data.Field(sequential=False, postprocessing=postproc)
IMDB_PATH='data/'
splits = torchtext.datasets.IMDB.splits(IMDB_TEXT, IMDB_LABEL, IMDB_PATH)
md2 = TextData.from_splits(PATH, splits, bs)

In [23]:
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
m3 = md2.get_model(opt_fn, 1500, bptt, bs, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           pad_token=pad_idx, dropout=0.3, dropouti=0.4, wdrop=0.3, dropoute=0.05, dropouth=0.2)
m3.model[0].trainable=False
m3.load_encoder(f'adam3_20_enc')

In [24]:
m3.fit(1e-3, 1, metrics=[accuracy])

A Jupyter Widget

[ 0.      0.2375  0.2052  0.9182]                            



In [25]:
m3[0].trainable=True
m3[0].encoder.trainable=False

In [26]:
m3.fit(2e-4, 2, metrics=[accuracy], reg_fn=reg_fn, cycle_len=1)

A Jupyter Widget

[ 0.      0.1894  0.1775  0.9328]                            
[ 1.      0.1831  0.1737  0.9338]                            



### End